In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

a = 2
b = 3
c = 4

print(f(a, b, c))

6.336362190988558


In [2]:
# caluclate gradient using calculus
def gradf(a, b, c):
  return [-3.0 * a**2 - 1/(2 * a**0.5), 3 * cos(3 * b) + 2.5*b**1.5, 1.0/c**2 ] # [df/da, df/db, df/dc]

# expected answers
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(a, b, c)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# estimate the gradient numerically without any calculus
h = 0.000001

f_0 = f(a, b, c)
dfda = (f(a + h, b, c) - f_0) / h
dfdb = (f(a, b + h, c) - f_0) / h
dfdc = (f(a, b, c + h) - f_0) / h

numerical_grad = [dfda, dfdb, dfdc] 

for dim in range(3):
  delta = abs(numerical_grad[dim] - ans[dim])
  ok = 'OK' if delta < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]} (delta: {delta})")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995 (delta: 5.958216721779763e-06)
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482 (delta: 1.3955669331267018e-06)
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534 (delta: 1.5256830465659732e-08)


In [4]:
# numerical gradients through symmetric derivatives (note that the approximation is quite a bit better!)

dfda_sym = (f(a + h, b, c) - f(a - h, b, c)) / (2*h)
dfdb_sym = (f(a, b + h, c) - f(a, b - h, c)) / (2*h)
dfdc_sym = (f(a, b, c + h) - f(a, b, c - h)) / (2*h)

numerical_grad2 = [dfda_sym, dfdb_sym, dfdc_sym]

for dim in range(3):
  delta = abs(numerical_grad2[dim] - ans[dim])
  ok = 'OK' if delta < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]} (delta: {delta})")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245 (delta: 7.599716411732516e-10)
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572 (delta: 2.9031710369054053e-09)
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188 (delta: 2.8629187909245957e-10)


## Section 2

In [13]:
from math import exp, log

class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  def __radd__(self, other):
    return self + other
  
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other):
    return self * other
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
    
  def __truediv__(self, other):
    return self * other**-1
  
  def __neg__(self):
    return self * -1

  def exp(self):
    out = Value(exp(self.data), (self,), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out
  
  def log(self):
    assert self.data > 0
    out = Value(log(self.data), (self,), 'log')

    def _backward():
      self.grad += (1.0/self.data) * out.grad
    out._backward = _backward

    return out

  def backward(self): # exactly as in video  
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [14]:
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

In [15]:
# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [31]:
import torch

logits_torch = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs_torch = torch.nn.functional.softmax(logits_torch, dim=0) # same result as our softmax
loss_torch = -torch.log(probs_torch[3])
loss_torch.backward()

print(f"{loss_torch.item()}")

for i, grad in enumerate(logits_torch.grad):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {i}: torch {grad.item():.16f}, expected  {ans[i]:.16f}")

2.1755154132843018
OK for dim 0: torch 0.0417725667357445, expected  0.0417725705153504
OK for dim 1: torch 0.8390244841575623, expected  0.8390245074625319
OK for dim 2: torch 0.0056533026508987, expected  0.0056533026622163
OK for dim 3: torch -0.8864504098892212, expected  -0.8864503806400986
